In [ ]:
# !pip install -U iminizinc

In [ ]:
# Import the necessary libraries
import os
%reload_ext iminizinc
from utils import import_instances, plot_solution

## Data Import

In [ ]:
# Get all instances from text files
instances = import_instances('input/instances/')
print(f'Number of instances: {len(instances)}')

## Constraint Programming

In [ ]:
NUMBER = 4

In [ ]:
# Get the number of blocks
BLOCKS = []
for n in range(int(instances[NUMBER][1])):
    BLOCKS.append(f'Block {n}')

print(f'Blocks: {BLOCKS}')

In [ ]:
# Get block lengths and heights
BLOCK_WIDTHS = []
BLOCK_HEIGHTS = []

for value in instances[NUMBER][2:]:
    width, height = value.split(' ')
    BLOCK_WIDTHS.append(int(width))
    BLOCK_HEIGHTS.append(int(height))
    
print(f'Block widths: {BLOCK_WIDTHS}')
print(f'Block heights: {BLOCK_HEIGHTS}')

In [ ]:
# Get the maximum width and height
MAX_WIDTH = int(instances[NUMBER][0])
MAX_HEIGHT = sum(BLOCK_HEIGHTS)

print(f'Maximum width: {MAX_WIDTH}')
print(f'Maximum height: {MAX_HEIGHT}')

In [ ]:
%%minizinc -m bind

include "globals.mzn";

enum BLOCKS;
array[BLOCKS] of int: BLOCK_WIDTHS;
array[BLOCKS] of int: BLOCK_HEIGHTS;

int: MAX_WIDTH;
int: MAX_HEIGHT;

% Constraints to find y-coordinates
array[BLOCKS] of var 0..MAX_HEIGHT: start_y;
var 0..MAX_HEIGHT: end_y;

constraint cumulative(start_y, BLOCK_HEIGHTS, BLOCK_WIDTHS, MAX_WIDTH);

constraint forall(b in BLOCKS)(start_y[b] + BLOCK_HEIGHTS[b] <= end_y);

% Constraints to find x-coordinates
array[BLOCKS] of var 0..MAX_WIDTH: start_x;

constraint cumulative(start_x, BLOCK_WIDTHS, BLOCK_HEIGHTS, end_y);

constraint forall(b in BLOCKS)(start_x[b] + BLOCK_WIDTHS[b] <= MAX_WIDTH);

solve minimize end_y;

In [ ]:
# Output
print(f'Minimum height: {end_y}')
print(f'Location x: {start_x}')
print(f'Location y: {start_y}')

In [ ]:
# Plot the solution found
circuits = []
for i in range(len(BLOCKS)):
    circuits.append([BLOCK_WIDTHS[i], BLOCK_HEIGHTS[i], start_x[i], start_y[i]])
    
plot_solution(MAX_WIDTH, end_y, len(BLOCKS), circuits)

## Data Output

In [ ]:
# Append the solution to the initial details
solution = []
for i, x in enumerate(instances[NUMBER]):
    if i == 0:
        solution.append(x + ' ' + str(end_y))
    elif i == 1:
        solution.append(x)
    else:
        solution.append(x + ' ' + str(start_x[i-2]) + ' ' + str(start_y[i-2]))
        
print(solution)

In [ ]:
# Write the solution to a text file
with open(f'output/cp/out-{NUMBER}.txt', 'w') as output:
    for item in solution:
        output.write(item)
        output.write('\n')